In [ ]:
!pip -qq install indic-nlp-library
!pip -qq install langdetect nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -qq catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Language detection with pretained langdect

In [ ]:
from langdetect import detect
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def split_text_by_language(text):
    sections = {}

    #clean the text:
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    text=re.sub(pattern, '', text)

    # Tokenize the text
    words = word_tokenize(text)

    # Use NLTK stopwords for English
    english_stopwords = set(stopwords.words('english'))

    current_language = None
    current_section = []

    for word in words:
        # Detect language for each word
        try:
          print(word)
          word_language = detect(word)

          if current_language is None:
              current_language = word_language

          # If the language changes, start a new section
          if current_language != word_language:
              sections[current_language] = ' '.join(current_section)
              current_section = []
              current_language = word_language

          # Exclude English stopwords
          if word_language != 'en' or word.lower() not in english_stopwords:
              current_section.append(word)
        except:
          continue

        # Add the last section
        sections[current_language] = ' '.join(current_section)

    return sections

# Example usage
text = "Hola, how are you doing today? C'est la vie. Привет, как дела? This is an example text."

language_sections = split_text_by_language(text)

for lang, section in language_sections.items():
    print(f"{lang}: {section}")

Hola
how
are
you
doing
today
Cest
la
vie
This
is
an
example
text
tr: Hola
so: today
ro: are
sw: you
pt: doing
fr: Cest
es: la
lv: vie
en: text
af: is
tl: an
ca: example


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from langdetect import detect
from nltk import word_tokenize
from nltk.corpus import stopwords
from indicnlp.tokenize import indic_tokenize

def split_text_by_language(text):
    sections = {}

    #clean the text:
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    text=re.sub(pattern, '', text)

    # Tokenize the text using Indic NLP for Indian languages
    words = indic_tokenize.trivial_tokenize(text)

    # Use NLTK stopwords for English
    english_stopwords = set(stopwords.words('english'))

    # Use custom stopwords for Kannada, Malayalam, and Tamil
    kannada_stopwords = set(['ನಾನು', 'ನೀನು', 'ಅವನು', 'ಅವಳು'])  # Add more as needed
    malayalam_stopwords = set(['ഞാൻ', 'നിനക്ക്', 'അവന്', 'അവള്'])  # Add more as needed
    tamil_stopwords = set(['நான்', 'நீ', 'அவன்', 'அவள்'])  # Add more as needed

    current_language = None
    current_section = []

    for word in words:
        # Detect language for each word
        word_language = detect(word)

        if current_language is None:
            current_language = word_language

        # If the language changes, start a new section
        if current_language != word_language:
            sections[current_language] = ' '.join(current_section)
            current_section = []
            current_language = word_language

        # Exclude stopwords based on language
        if word_language == 'en' and word.lower() in english_stopwords:
            continue
        elif word_language == 'kn' and word in kannada_stopwords:
            continue
        elif word_language == 'ml' and word in malayalam_stopwords:
            continue
        elif word_language == 'ta' and word in tamil_stopwords:
            continue

        current_section.append(word)

    # Add the last section
    sections[current_language] = ' '.join(current_section)

    return sections

# Example usage
text = "ನಮಸ್ಕಾರ, സുപ്രഭാതം, வணக்கம். How are you today? ശുഭ രാത്രി."

language_sections = split_text_by_language(text)

for lang, section in language_sections.items():
    print(f"{lang}: {section}")

pl: How
ro: are
sw: you
so: today


## Combination of models

In [ ]:
t_model="/content/drive/MyDrive/Capstone/Models/Tamil/muril_tamil.sav"
m_model="/content/drive/MyDrive/Capstone/Models/Malayalam/muril_malayalam.sav"
k_model="/content/drive/MyDrive/Capstone/Models/Kannada/muril_kannada.sav"

In [ ]:
!pip -qq install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 95.

In [ ]:
import pickle
# load the model from disk
t_model_model = pickle.load(open(t_model, 'rb'))
m_model_model =pickle.load(open(m_model,'rb'))
k_model_model=pickle.load(open(k_model,'rb'))

In [31]:
label=["Not_Hate", "Hateful", "Not-Malayalam"]

languages=['tamil','malayalam','kannada']

#class labels dict
class_dict={}

for i in languages:
  d=dict()
  for j in range(3):
    if j == 2:
      d[j]=label[j].split("-")[0]+"-"+i
    else:
      d[j]=label[j]
  class_dict[i]=d

class_dict

{'tamil': {0: 'Not_Hate', 1: 'Hateful', 2: 'Not-tamil'},
 'malayalam': {0: 'Not_Hate', 1: 'Hateful', 2: 'Not-malayalam'},
 'kannada': {0: 'Not_Hate', 1: 'Hateful', 2: 'Not-kannada'}}

In [ ]:
!pip -qq install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
!pip -qq install langdetect

In [ ]:
!pip -qq install tqdm

In [66]:
import re
import string
from IPython.display import clear_output
from googletrans import Translator
from langdetect import detect
from tqdm import tqdm


def split_text_language(text, target_language='ta'):
    # specify the target_language
    if target_language.lower() == "tamil":
        target_language = "ta"
    elif target_language.lower() == "malayalam":
        target_language = "ml"
    else:
        target_language = "kn"

    # clean the text
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')

    tamil_words, kannada_words, malayalam_words, other_words = [], [], [], []

    sources=[]

    for word in tqdm(text.split(" ")):
        flag = False
        try:
            source_language = detect(word)
        except:
            source_language = None

        sources.append(source_language)



        # check if the word is Tamil
        if t_model_model.predict([word])[0][0] != 2 or source_language=='ta':
            tamil_words.append(word)
            flag = True
        # checking for Kannada word
        if k_model_model.predict([word])[0][0] != 2 or source_language=='kn':
            kannada_words.append(word)
            flag = True
        # checking for Malayalam word
        if m_model_model.predict([word])[0][0] != 2 or source_language=='ml':
            malayalam_words.append(word)
            flag = True
        # if the words don't belong to these three languages
        if not flag:
            # Translate the word into the target language
            if source_language:
                translator = Translator()
                translated_text = translator.translate(word, src=source_language, dest=target_language).text
                other_words.append(translated_text)
            else:
                other_words.append(word)

    clear_output()


    print(other_words)

    print("Sources=", sources)

    # creating the language of words dictionary
    language_words = {}
    if len(tamil_words) != 0:
        language_words['Tamil_words'] = ' '.join(tamil_words)
    if len(malayalam_words) != 0:
        language_words['Malayalam_words'] = ' '.join(malayalam_words)
    if len(kannada_words) != 0:
        language_words['Kannada_words'] = ' '.join(kannada_words)
    if len(other_words) != 0:
        language_words['Other_words'] = ' '.join(other_words)

    return language_words

#split the words based on languages

#specify the target language argument to convert the other language words into required language words (tamil/kannada/malayalam)
split_text_language("ಮಿಸ್ ಎಲಾ ಮಾಡ್ಕೊಳೋಲ್ಲ ಬಿಡಿ ಇ ಟೆಲಿಗ್ರಾಂನಲ್ಲಿ ಡೌನ್‌ಲೋಡ್ ಮಾಡಬಹುದು ಅಥವಾ ಸ್ವಲ್ಪ ದೇನಾ ಒದ್ರೆ ನಲ್ಲೆ ಭರುತೆ",
                    target_language="tamil")

[]
Sources= ['kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn']


{'Tamil_words': 'ಮಿಸ್ ಎಲಾ ಮಾಡ್ಕೊಳೋಲ್ಲ ಬಿಡಿ ಇ ಟೆಲಿಗ್ರಾಂನಲ್ಲಿ ಡೌನ್\u200cಲೋಡ್ ಮಾಡಬಹುದು ಅಥವಾ ಸ್ವಲ್ಪ ದೇನಾ ಒದ್ರೆ ನಲ್ಲೆ ಭರುತೆ',
 'Malayalam_words': 'ಮಿಸ್ ಎಲಾ ಮಾಡ್ಕೊಳೋಲ್ಲ ಬಿಡಿ ಇ ಡೌನ್\u200cಲೋಡ್ ಮಾಡಬಹುದು ಅಥವಾ ಸ್ವಲ್ಪ ದೇನಾ ಒದ್ರೆ ನಲ್ಲೆ ಭರುತೆ',
 'Kannada_words': 'ಮಿಸ್ ಎಲಾ ಮಾಡ್ಕೊಳೋಲ್ಲ ಬಿಡಿ ಇ ಟೆಲಿಗ್ರಾಂನಲ್ಲಿ ಡೌನ್\u200cಲೋಡ್ ಮಾಡಬಹುದು ಅಥವಾ ಸ್ವಲ್ಪ ದೇನಾ ಒದ್ರೆ ನಲ್ಲೆ ಭರುತೆ'}

## Prediction from the 3 models

In [67]:
def prediction(model,text): #sample testing
  return model.predict([text])[0]
res=prediction(m_model_model,'ಮಿಸ್ ಎಲಾ ಮಾಡ್ಕೊಳೋಲ್ಲ ಬಿಡಿ ಇ ಟೆಲಿಗ್ರಾಂನಲ್ಲಿ ಡೌನ್‌ಲೋಡ್ ಮಾಡಬಹುದು ಅಥವಾ ಸ್ವಲ್ಪ ದೇನಾ ಒದ್ರೆ ನಲ್ಲೆ ಭರುತೆ')
print(class_dict['malayalam'][res[0]])

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Hateful


In [47]:
def prediction(pred): #testing with combination of models
  prediction={}
  for i in pred.keys():
    if i=="Tamil_words":
      out=t_model_model.predict([pred[i]])
      #classifier

      prediction['Tamil']=[class_dict[i.split("_")[0].lower()][out[0][0]],out[1][0][out[0][0]]]
    elif i=="Kannada_words":
      out=k_model_model.predict([pred[i]])
      prediction['Kannada']=class_dict[i.split("_")[0].lower()][out[0][0]],out[1][0][out[0][0]]
    elif i=="Malayalam_words":
      out=m_model_model.predict([pred[i]])
      prediction['Malayalam']=class_dict[i.split("_")[0].lower()][out[0][0]],out[1][0][out[0][0]]

  clear_output()
  return prediction

text="A multilingual movie review featuring Prabhas and Anushka Shetty, portraying love and societal constructs . fans la oru loosa irupanka polla"
language_dict=split_text_language(text,target_language="tamil") #convert the other words into tamil
result_dict=prediction(language_dict)
for lang,out in result_dict.items():
  print(f"The words in {lang} belongs to {out[0]} category with the score from model output as {out[1]}")


The words in Tamil belongs to Not_Hate category with the score from model output as 3.502986431121826
The words in Malayalam belongs to Not_Hate category with the score from model output as 4.29678201675415
The words in Kannada belongs to Not_Hate category with the score from model output as 3.1601922512054443


### Prediction with the classifier (catboost)

In [44]:
#load the model

#model path
from catboost import CatBoostClassifier

catboost_tamil="/content/drive/MyDrive/Capstone/Models/Classifiers/tamil_catboost_muril_classifier.sav"
catboost_kannada="/content/drive/MyDrive/Capstone/Models/Classifiers/kannada_catboost_muril_classifier.sav"
catboost_malayalam="/content/drive/MyDrive/Capstone/Models/Classifiers/malayalam_catboost_muril_classifier.sav"

import pickle
# load the model from disk
catboost_tamil = pickle.load(open(catboost_tamil, 'rb'))

catboost_kannada = pickle.load(open(catboost_kannada, 'rb'))

catboost_malayalam = pickle.load(open(catboost_malayalam, 'rb'))

In [59]:
def prediction(model,text): #sample testing
  return catboost_malayalam.predict(([model.predict([text])[1][0]]))
res=prediction(m_model_model,'aaa dislike adicha poorimakkalude ammakku mohanlal anu adichu kodukkunnadu tonnunnu')
print([res])

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[array([[1]])]


In [36]:
def prediction(pred): #testing with combination of models
  prediction={}
  for i in pred.keys():
    if i=="Tamil_words":
      out=t_model_model.predict([pred[i]])[1][0]
      #classifier
      out=catboost_tamil.predict([out])
      prediction['Tamil']=class_dict[i.split("_")[0].lower()][out[0][0]]
    elif i=="Kannada_words":
      out=k_model_model.predict([pred[i]])
      prediction['Kannada']=class_dict[i.split("_")[0].lower()][out[0][0]]
    elif i=="Malayalam_words":
      out=m_model_model.predict([pred[i]])
      prediction['Malayalam']=class_dict[i.split("_")[0].lower()][out[0][0]]

  clear_output()
  return prediction

text="ಪ್ರೇಮತ್ತಿನ ಚಿರಕೆ, പ്രണയത്തിന്റെ ചിറക്, காதலின் ப: A multilingual movie review featuring Prabhas and Anushka Shetty, portraying love and societal constructs"
language_dict=split_text_language(text,target_language="tamil") #convert the other words into tamil
result_dict=prediction(language_dict)
for lang,out in result_dict.items():
  print(f"The words in {lang} belongs to {out[0]} category")



[]


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.